# Self-Driving Car Engineer Nanodegree

## Computer Vision
## Project: Vehicle Detection and Tracking
---

#### All imports

In [1]:
import numpy as np
import cv2
import glob
from skimage.feature import hog
from skimage import color, exposure
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split
from IPython.display import HTML
from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label
import matplotlib.image as mpimg

#### All functions

In [2]:
def reader(root_dir):
    list_out = []
    for subdir in (os.listdir(root_dir)):
        for file in os.listdir(os.path.join(root_dir, subdir)):
            img = mpimg.imread(os.path.join(root_dir, subdir, file))
            feature = single_img_features(img)
            list_out.append(feature)
    return list_out

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """return HOG features and visualization"""
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    '''compute binned color features '''
    # create the feature vector
    features = cv2.resize(img, size).ravel() 
    return features

def color_hist(img, nbins=32):
    '''compute color histogram features '''
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return hist_features

def single_img_features(img, spatial_size=(32, 32), hist_bins=32, orient=12, pix_per_cell=8, cell_per_block=1, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], orient, pix_per_cell,
                                                     cell_per_block, vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, pix_per_cell,
                                            cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)     
    return np.concatenate(img_features)

# Define a function that takes an image, start and stop positions in both x and y,  window size (x and y dimensions),   and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions Note: you could vectorize this step, but in practice you'll be considering windows one by one with your classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

def search_windows(img, windows, clf, scaler):
    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
        features = single_img_features(test_img)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    return on_windows

def add_heat(heatmap, bbox_list):
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

#### Read training data

In [3]:
car_features = reader('vehicles')
notcar_features = reader('non-vehicles')
print('car_features length=', len(car_features), 'notcar_features length=', len(notcar_features), notcar_features[0].shape)

car_features length= 8792 notcar_features length= 8968 (3936,)


#### Prepare training data for the classifier

In [4]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
X_scaler = StandardScaler().fit(X)
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=np.random.randint(0, 100))

del car_features
del notcar_features
del X
del y
del scaled_X

#### Train classifier

In [5]:
# Use a linear SVC 
svc = LinearSVC(C = 0.001)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

3.97 Seconds to train SVC...
Test Accuracy of SVC =  0.991


#### Define Pipeline for video

In [6]:
def pipeline(img_in):
    
    # Normalize the data to 0.0-1.0 to fit with the training data
    img = img_in.astype(np.float32) / 255
    
    windows = slide_window(img, x_start_stop=[None,None], y_start_stop=[390, 560], xy_window=(70,70), xy_overlap=(0.7,0.7))

    hot_windows = search_windows(img, windows, svc, X_scaler)                       

    heat = np.zeros_like(img_in[:,:,0]).astype(np.float)

    # Add heat to each box in box list
    heat = add_heat(heat,hot_windows)
    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)
    
    # Find final boxes from heatmap using label function
    labels = label(np.clip(heat, 0, 255))
    draw_img = draw_labeled_bboxes(np.copy(img_in), labels)

    return draw_img

#### Process video

In [7]:
vid_out = 'video_out.mp4'
vid = VideoFileClip('project_video.mp4')
vid_clip = vid.fl_image(pipeline)
%time vid_clip.write_videofile(vid_out, audio=False)

HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(vid_out))

[MoviePy] >>>> Building video video_out.mp4
[MoviePy] Writing video video_out.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [13:56<00:00,  1.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video_out.mp4 

Wall time: 13min 58s
